# Day 10: Cheminformatics

Morning! I've got some fun demos to go through to explain how to use LLMs, and as well machine learning in chemistry applications.

The first thing I have for you is a demo to help visualize the concept of embeddings. Any information, whether it be words, an image, or a pdb file, can be reduced in dimensionality to some extent of accuracy. What do I mean by reduced in dimensionality? Here's an image to help grasp the concept:

In [ ]:
import matplotlib.pyplot as plt
# Plot the image
plt.show(plt.imshow(plt.imread('auto-encoder.png')))

This example is a diagram showing how a particular neural architecture, the auto-encoder, functions. In reality, the actual simulated network would likely have far more neurons, but this picture serves as an example. 

As you can see, the input and output layers are the same size, but the middle 'embedding layer' is made of less neurons. This means that the number of numbers needed to describe the input is reduced, and then the rest of the layers of neurons after this attempt to reconstruct the original information using only the reduced set of embeddings. 

This is how our brains work too! We operate by reducing the raw information we receive (excitations in the electrons in the photoreceptive proteins in our eyes) into a set of embeddings about the information (that's a dog). So, the concept of embeddings is essentially equivalent to the concept of concepts! All ideas as we know them are embeddings derived from information we have perceived. 

I'd like to take a moment to compare the idea of embeddings to the ideas proposed by people like Plato in the Allegory of the Cave, as well as Jacques Derrida in his theory of deconstuctionism. Embeddings are the shadow of reality that we experience that Plato spoke of in his allegory, and the space of relation created by embeddings is the space of meaning proposed in deconstructionism. These two were concerned with spirituality and poetry respectively, but now that we've advanced far enough mathematically, we can actually quantify our experience of reality and meaning in a way that would probably make them incredibly happy, were they still alive to see it.

Knowing this, we can describe and understand the behavior of AI models a lot more easily, as we will see in the later examples. For now, let's get into this demo helping to explain specifically word embeddings. We'll need to start by making sure we have the required libraries to run the demo.

In [ ]:
!pip install matplotlib gensim sklearn numpy rdkit transformers torch torchtext==0.4.0

In [ ]:
import gensim.downloader as api
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import plotly.graph_objects as go
import nltk
import random
from sklearn.neighbors import NearestNeighbors
import logging
import ipywidgets as widgets
from ipywidgets import IntSlider, Checkbox, Button, HBox, VBox
import IPython.display as display
import gzip
import pandas as pd

Great! Now we can actually run the program. Refer to the comments in the code to see how exactly everything is working.

# !!!Warning! The random words, max word embeddings, and nerest neighbor search tool may produce words that are offensive or generally upsetting, turn on at your own risk!!!

What we're doing here is training a model to learn the relationships between words based on context. The way the parameters for this are set up currently is to make this possible on relatively basic hardware. For real applications, you'd want to train it for far longer, and likely run an optimization on the training parameters. But for this demo, a quick training and visualization should be sufficient.

The first thing to do is to define the training variables. Let's go through these:

In [ ]:
# Define fun variables to play with, all set relatively low by default for time's sake

num_sentences = 10000000 # We are using a dataset of wikipedia articles for this, and this is the number of sentences
# from the dataset that we will be using. Feel free to set this higher or lower, just know it impacts memory usage.

dimensions = 24 # This is what we were just talking about with embeddings. This is the number of neurons that will be
# in the embedding layer of the neural network. Essentially, this is the number of distinct values that will be used to
# distinguish between words. Note that in the representation, the words will be mapped around a sphere, so a lot of
# dimensionality is lost, however we still want to train in more dimensions to accurately distinguish between words.

number_of_embeddings = dimensions # This is meant as a point of clarity, that embeddings and dimensions mean the same
# thing in this context.

epochs = 10 # This is the number of training cycles the model will do, set low for time's sake.

# Define the set of words to graph
common_nouns = set([
    "time", "year", "people", "way", "day", "man", "thing", "woman", "life", "child",
    "world", "school", "state", "family", "student", "group", "country", "problem", "hand", "part",
    "place", "case", "week", "company", "system", "program", "question", "work", "government", "number",
    "night", "point", "home", "water", "room", "mother", "area", "money", "story", "fact",
    "month", "lot", "right", "study", "book", "eye", "job", "word", "business", "issue",
    "side", "kind", "head", "house", "service", "friend", "father", "power", "hour", "game",
    "line", "end", "member", "law", "car", "city", "community", "name", "president", "team",
    "minute", "idea", "kid", "body", "information", "back", "parent", "face", "others", "level",
    "office", "door", "health", "person", "art", "war", "history", "party", "result", "change",
    "morning", "reason", "research", "girl", "guy", "moment", "air", "teacher", "force", "education"
])

Now that we've defined our variables, we'll start by defining the Word2Vec model and initiate the training process.

In [ ]:
# Enable logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Load the text8 dataset
dataset = api.load('text8')
dataset = list(dataset)
dataset = dataset[:num_sentences]

# Train a Word2Vec model
print("Training Word2Vec model...")
model = Word2Vec(dataset, vector_size=number_of_embeddings, window=5, min_count=1, workers=4, epochs=epochs)


Here, we are initializing a Word2Vec model and training it on the 'text8' dataset. Word2Vec is a two-layer neural network that processes text by "vectorizing" words. Its input is a text corpus and its output is a set of vectors, also known as feature vectors or word embeddings. The purpose of Word2Vec is to create a numeric representation of words that preserves the contextual similarity of words. Words that are used in similar ways tend to be closer to each other in the vector space.

The parameters of the Word2Vec model are:

dataset: The sentences iterable to be used for training the model.
vector_size: Dimensionality of the word vectors.
window: Maximum distance between the current and predicted word within a sentence.
min_count: Ignores all words with total frequency lower than this.
workers: Use these many worker threads to train the model.
epochs: Number of iterations (epochs) over the corpus.

Next, we define the GUI elements to interact with the model and visualize the embeddings.

In [ ]:
# Create an IntSlider to set the number of random words to include
num_random_words_slider = IntSlider(min=0, max=100, step=1, value=0, description='Random Words')

# Create a Checkbox to decide whether to include the dictionary of common nouns
common_nouns_checkbox = Checkbox(value=True, description='Include Common Nouns')

# Create a Checkbox to decide whether to include the maximum vectors in each dimension
max_vectors_checkbox = Checkbox(value=False, description='Include Max Vectors')

# Create a Button to run the training for another epoch and update the graph with the current words
train_button = Button(description='Train for Another Epoch')
train_button.on_click(lambda b: train_button_clicked())

# Create a Button to regenerate the graphs without re-randomizing
regenerate_button = Button(description='Regenerate Graphs')
regenerate_button.on_click(lambda b: regenerate_button_clicked())

# Create a Button to regenerate with new random words
randomize_button = Button(description='Regenerate with New Random Words')
randomize_button.on_click(lambda b: randomize_button_clicked())

These widgets will allow the user to interact with the model, change its parameters, and visualize the embeddings in different ways. The sliders and checkboxes allow the user to control the number of words visualized, whether to include common nouns and maximum vectors. The buttons are used to retrain the model, regenerate the graphs with the same words, or regenerate the graphs with new random words.

Next, we'll define the function train_button_clicked, which will train the model for another epoch when the "Train for Another Epoch" button is clicked. This function updates the word vectors and regenerates the graphs to reflect the new embeddings.

In [ ]:
def train_button_clicked():
    # Train the model for another epoch
    model.train(dataset, total_examples=len(dataset), epochs=1)
    # Update the word vectors
    word_vectors = model.wv
    # Regenerate the graphs
    regenerate_graphs()

Similarly, we define regenerate_button_clicked and randomize_button_clicked functions. These functions are called when the corresponding buttons are clicked.

In [ ]:
def regenerate_button_clicked():
    # Regenerate the graphs
    regenerate_graphs()

def randomize_button_clicked():
    # Randomly select words to visualize
    words_to_visualize = random.sample(word_vectors.index_to_key, num_random_words_slider.value)
    # Regenerate the graphs
    regenerate_graphs()

The regenerate_button_clicked function simply calls regenerate_graphs to regenerate the graphs without re-randomizing the words. The randomize_button_clicked function selects a random set of words from the word vectors according to the value of the num_random_words_slider, and then regenerates the graphs.

We also need to define the create_3d_plot function before it's used in the regenerate_graphs function. This function creates a 3D plot to visualize the word embeddings. It plots embeddings for common words, random words, and optionally maximum words.

In [ ]:
def create_3d_plot(common_words, random_words, x_3d_common, y_3d_common, z_3d_common, x_3d_random, y_3d_random, z_3d_random, max_words=None):
    # Create an interactive 3D plot
    print("Creating 3D plot...")
    fig_3d = go.Figure()

    # Add the spherical plot of word embeddings for common nouns from the dictionary set below
    fig_3d.add_trace(go.Scatter3d(
        x = x_3d_common,
        y = y_3d_common,
        z = z_3d_common,
        mode = 'markers+text',
        text = common_words,
        textposition = 'top center',
        name = 'Common Word Embeddings',
        marker=dict(
            color='blue',  # set color to blue
            opacity=1.0
        )
    ))

    # Add the spherical plot of word embeddings for random words
    fig_3d.add_trace(go.Scatter3d(
        x = x_3d_random,
        y = y_3d_random,
        z = z_3d_random,
        mode = 'markers+text',
        text = random_words,
        textposition = 'top center',
        name = 'Random Word Embeddings',
        marker=dict(
            color='red',  # set color to red
            opacity=1.0
        )
    ))

    # Add the spherical plot of maximum word embeddings to the 3D plot
    if max_words is not None:
        fig_3d.add_trace(go.Scatter3d(
            x = max_words['x'],
            y = max_words['y'],
            z = max_words['z'],
            mode = 'markers+text',
            text = max_words['words'],
            textposition = 'top center',
            name = 'Max Word Embeddings'
        ))

    # Add the unit sphere, but make it slightly smaller so we can see the words easier
    phi = np.linspace(0, 2*np.pi, 100)
    theta = np.linspace(0, np.pi, 100)
    x_sphere = 0.95 * np.outer(np.cos(phi), np.sin(theta))
    y_sphere = 0.95 * np.outer(np.sin(phi), np.sin(theta))
    z_sphere = 0.95 * np.outer(np.ones(np.size(phi)), np.cos(theta))

    fig_3d.add_trace(go.Surface(
        x = x_sphere,
        y = y_sphere,
        z = z_sphere,
        opacity = 1.0,
        showscale = False,
        name = 'Unit Sphere'
    ))

    fig_3d.update_layout(width=800, height=800, title_text="Word Embeddings Visualized on a Sphere", scene=dict(bgcolor='white'))
    
    fig_3d.show()


This function creates a 3D scatter plot with word embeddings for common words and random words. The embeddings are plotted on a unit sphere to help visualize their relative positions in the vector space. If provided, the embeddings with maximum values in each dimension are also plotted.

The create_3d_plot function is quite complex and contains a lot of code related to the creation of the 3D plot, such as defining the markers for the scatter plot, setting the colors and opacities, and creating a unit sphere.

Now, we are going to define the regenerate_graphs function. This function is responsible for creating and updating the visualizations of the word embeddings. This includes selecting the words to visualize, transforming the embeddings into a format that can be visualized, and generating the 3D plot.

We'll start by setting up the words we want to visualize. We have common words, random words, and the maximum words, each of which are controlled by the GUI elements we defined earlier. We first clear the current output to prepare for the new visualizations. We then fetch the word vectors from the trained model and select the words we want to visualize. If the "Include Common Nouns" checkbox is checked, we include the common words in our visualization. If the slider for random words is set to a value greater than zero, we include that many random words.

Next, we need to handle the case where we don't have any words to visualize. If there are no words to visualize (neither common words nor random words are selected), we display a message indicating that no words are selected for visualization and show the GUI elements to allow the user to select words.

Now that we have the words we want to visualize, we need to retrieve their corresponding vectors from the trained model. We'll separate the common words and the random words. Here we're extracting the word vectors corresponding to the common words and random words, then combining them into a single numpy array. We need to do this because the t-SNE algorithm, which we'll use later to reduce the dimensionality of our vectors for visualization, operates on a single continuous block of data.

Next, we will calculate the t-SNE transformation for the vectors. t-SNE (t-Distributed Stochastic Neighbor Embedding) is a technique for dimensionality reduction that is particularly well suited for the visualization of high-dimensional datasets. It helps us to visualize high-dimensional data in two or three dimensions. We're using the t-SNE algorithm to reduce the dimensionality of our vectors from their original high-dimensional space (300 dimensions for Word2Vec) down to 3 dimensions so that we can visualize them. We normalize these values between -1 and 1 just for simplicity's sake. Then we split the words back into separate categories so we can color them differently on the graph.

Next, we calculate the spherical coordinates for our 3D vectors. Spherical coordinates are a system of coordinates that describes points in three-dimensional space using three parameters: the radial distance of the point from a fixed origin, its inclination angle measured from a fixed zenith direction, and the azimuth angle of its orthogonal projection on a reference plane that passes through the origin and is orthogonal to the zenith. We're computing the azimuth (theta) and inclination (phi) for each of our 3D vectors, separately for the common words and random words. This will allow us to plot our word vectors on a sphere, which can give us a better idea of their relative positions in the high-dimensional space.

We now calculate the words with the maximum values in each dimension of the embeddings, if the checkbox for this option is selected in the GUI. This is meant to provide intuition about how exactly the model is learning the difference between words. By looking at the most highly valued word in each individual dimension, we can get a sense of the ideas the dimensions correspond to.

In [ ]:
def regenerate_graphs():
    # Clear the current output
    display.clear_output(wait=True)
    
    # Get the word vectors
    word_vectors = model.wv

    # Select words to visualize
    words_to_visualize = word_vectors.index_to_key

    # Select common nouns using dictionary
    print("Selecting words...")
    selected_words = [word for word in words_to_visualize if word in common_nouns]

    # Get the vectors/embeddings for the selected words
    vectors_to_visualize = [word_vectors[word] for word in selected_words]

    # Convert list to numpy array
    vectors_to_visualize = np.array(vectors_to_visualize)

    # Select words to visualize
    common_words = []
    random_words = []
    if common_nouns_checkbox.value:
        common_words = [word for word in words_to_visualize if word in common_nouns]
    if num_random_words_slider.value > 0:
        random_words = random.sample(words_to_visualize, num_random_words_slider.value)
        
    # If there are no words to visualize, return
    if not common_words and not random_words:
        print("No words to visualize. Please select at least one option.")
        # Display the widgets
        display.display(VBox([HBox([num_random_words_slider, common_nouns_checkbox, max_vectors_checkbox]),
                      HBox([train_button, regenerate_button, randomize_button])]))
        return
    
    # Get the vectors/embeddings for the selected words
    vectors_common = [word_vectors[word] for word in common_words]
    vectors_random = [word_vectors[word] for word in random_words] if random_words else []

    # Combine the vectors
    vectors_combined = vectors_common + vectors_random

    # Convert to np array
    vectors_combined = np.array(vectors_combined)
    
    # Compute the t-SNE transformation for 3D
    print("Computing t-SNE...")
    tsne = TSNE(n_components=3, random_state=42, perplexity=min(30, len(vectors_combined)-1))
    vectors_3d_combined = tsne.fit_transform(vectors_combined)

    # Normalize the output to be between -1 and 1
    scaler = MinMaxScaler(feature_range=(-1, 1))
    vectors_3d_combined = scaler.fit_transform(vectors_3d_combined)
    
    # Split the combined vectors back into common and random vectors
    vectors_3d_common = vectors_3d_combined[:len(vectors_common)]
    vectors_3d_random = vectors_3d_combined[len(vectors_common):]

    # Compute the spherical coordinates
    if vectors_3d_common.ndim == 2:
        theta_common = np.arctan2(vectors_3d_common[:, 1], vectors_3d_common[:, 0])
        phi_common = np.arccos(vectors_3d_common[:, 2])
    else:
        theta_common = []
        phi_common = []

    if vectors_3d_random.ndim == 2:
        theta_random = np.arctan2(vectors_3d_random[:, 1], vectors_3d_random[:, 0])
        phi_random = np.arccos(vectors_3d_random[:, 2])
    else:
        theta_random = []
        phi_random = []
        
    # Map the words onto a sphere for 3D plot
    x_3d_common = np.sin(phi_common) * np.cos(theta_common)
    y_3d_common = np.sin(phi_common) * np.sin(theta_common)
    z_3d_common = np.cos(phi_common)
    x_3d_random = np.sin(phi_random) * np.cos(theta_random) if len(theta_random) > 0 else []
    y_3d_random = np.sin(phi_random) * np.sin(theta_random) if len(theta_random) > 0 else []
    z_3d_random = np.cos(phi_random) if len(phi_random) > 0 else []

    # Initialize max_words as an empty dictionary
    max_words = {'words': [], 'x': [], 'y': [], 'z': []}
    
    # Find the words with the maximum value in each dimension of the embeddings
    if max_vectors_checkbox.value:
        print("Finding max vectors...")
        for dim in range(dimensions):
            max_word = word_vectors.index_to_key[np.argmax(vectors_combined[:, dim])]
            max_words['words'].append(max_word)
            max_words['x'].append(vectors_3d_combined[word_vectors.key_to_index[max_word], 0])
            max_words['y'].append(vectors_3d_combined[word_vectors.key_to_index[max_word], 1])
            max_words['z'].append(vectors_3d_combined[word_vectors.key_to_index[max_word], 2])

    # Compute the spherical coordinates for max words
    if max_words['x']:
        theta_max = np.arctan2(max_words['y'], max_words['x'])
        phi_max = np.arccos(max_words['z'])

        # Map the max words onto a sphere for 3D plot
        max_words['x'] = np.sin(phi_max) * np.cos(theta_max)
        max_words['y'] = np.sin(phi_max) * np.sin(theta_max)
        max_words['z'] = np.cos(phi_max)

    # Create a 3D plot of the word embeddings
    create_3d_plot(common_words, random_words, x_3d_common, y_3d_common, z_3d_common, x_3d_random, y_3d_random, z_3d_random, max_words)
    
    # Display the widgets
    display.display(VBox([HBox([num_random_words_slider, common_nouns_checkbox, max_vectors_checkbox]),
                  HBox([train_button, regenerate_button, randomize_button])]))



That was a lot! I hope at least some if it made sense, but if not, let's visualize it and I'm sure that will help!

In [ ]:
regenerate_graphs()

In the final part of the code, we create an interactive nearest neighbors search. This is a valuable tool for exploring the word embeddings and understanding the relationships between different words.

Let's start by defining a function find_nearest_neighbors that finds the nearest neighbors to a given word in the embedding space.

In [ ]:
def find_nearest_neighbors(word, topn=5):
    # Find the nearest neighbors to the given word
    nearest_neighbors = model.wv.most_similar(word, topn=topn)
    # Return the nearest neighbors
    return nearest_neighbors


This function uses the most_similar method provided by the Word2Vec model to find the words that have the most similar embeddings to the given word. The topn parameter controls how many similar words to return.

Next, we create an interactive nearest neighbors search using IPython widgets.

In [ ]:
# Create a Text widget for input
word_input = widgets.Text(value='king', description='Word:')

# Create an IntSlider widget for the number of nearest neighbors
num_neighbors_slider = widgets.IntSlider(value=5, min=1, max=20, step=1, description='Neighbors:')

# Create a Button widget to perform the search
search_button = widgets.Button(description='Search')

# Display the widgets
display.display(word_input, num_neighbors_slider, search_button)

# Define a function to perform the search when the button is clicked
def on_search_button_clicked(b):
    print()
    # Get the input word and number of neighbors
    word = word_input.value
    print(f"Searching for words like {word}:")
    num_neighbors = num_neighbors_slider.value
    try:
        # Find the nearest neighbors
        nearest_neighbors = find_nearest_neighbors(word, topn=num_neighbors)
        # Print the nearest neighbors
        for neighbor, similarity in nearest_neighbors:
            print(f"{neighbor} is ~{int(similarity*100)}% similar")
    except KeyError:
        print("That word wasn't in the training vocabulary, please try a different word.")
        
# Attach the function to the button
search_button.on_click(on_search_button_clicked)


Here we're creating a text input field for the user to input a word, a slider to control the number of nearest neighbors to find, and a button to perform the search. When the button is clicked, the on_search_button_clicked function is called, which retrieves the word and number of neighbors from the widgets, finds the nearest neighbors using the find_nearest_neighbors function, and prints the results.

That's it for this one! Let's jump into the next one to see this exact same graph type applied specifically to chemicals!